# 3.4 Enrich dataset with news 



In [1]:
import pandas as pd

In [17]:
# show *all* rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# widen the “page” so it won’t wrap columns  
pd.set_option('display.width', 1000)

In [2]:
df_listings = pd.read_csv("./data/eustartup_listings_enriched_3.csv")

df_listings

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,...,redirected_url,name_cleaned,Startup,Instagram,X,LinkedIn,General,general_root,website_host,general_host
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,...,NaN,Avdain,Avdain,https://www.instagram.com/ns.rabbani/p/BSEJIDS...,https://x.com/awesomeindiehub/status/187271911...,NaN,https://www.avdain.com/our-mission,https://avdain.com,avdain.com,avdain.com
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,...,NaN,Popper Power,Popper Power,https://www.instagram.com/popperpower/,https://x.com/FranzHoegl/status/18789481448322...,https://at.linkedin.com/company/popperpower,https://www.popperpower.com/?srsltid=AfmBOoqrk...,https://popperpower.com,popperpower.com,popperpower.com
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,...,NaN,Setter AI,Setter AI,https://www.instagram.com/trysetter/,https://x.com/trysetter,https://www.linkedin.com/company/setterai?trk=...,https://www.trysetter.com/,https://trysetter.com,trysetter.com,trysetter.com
3,SurveySensum,https://www.eu-startups.com/directory/surveyse...,https://www.eu-startups.com/wp-content/uploads...,https://www.surveysensum.com,vienna,Vienna,"customer feedback, customer experience",Software & Analytics,SurveySensum is a leading customer feedback pl...,2018,...,NaN,SurveySensum,SurveySensum,https://www.instagram.com/surveysensum/,https://x.com/surveysensum,https://www.linkedin.com/company/surveysensum,https://www.surveysensum.com/,https://surveysensum.com,surveysensum.com,surveysensum.com
4,Artypa,https://www.eu-startups.com/directory/artypa/,https://www.eu-startups.com/wp-content/uploads...,https://artypa.com,Vienna,Vienna,"ai content, ai productivity, ai generation",Software & Analytics,Traditional AI workflows often involve navigat...,2024,...,NaN,Artypa,Artypa,https://www.instagram.com/artypa/,https://x.com/MicroLaunchHQ/status/18729305332...,NaN,https://artypa.com/,https://artypa.com,artypa.com,artypa.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
510,Ease-Link,https://www.eu-startups.com/directory/ease-link/,https://www.eu-startups.com/wp-content/uploads...,http://easelink.com/,Graz,Styria,"automotive, industrial automation",Mobility,"Ease-Link GmbH develops MATRIX CHARGING®, the ...",2015,...,NaN,Ease-Link,Ease-Link,https://www.instagram.com/safihome.lb/reel/DHq...,https://x.com/TrendingTopicsA/status/189288436...,https://de.linkedin.com/company/howtoabroad,https://easelink.com/,https://easelink.com,easelink.com,easelink.com
511,Healcloud,https://www.eu-startups.com/directory/healcloud/,https://www.eu-startups.com/wp-content/uploads...,http://www.healcloud.com/,Vienna,Vienna,"big data, health care",Software & Analytics,Healcloud is a European health tech enabler to...,2015,...,NaN,Healcloud,Healcloud,https://www.instagram.com/healcloudcafe.atpua/,https://x.com/Paloandtogo/status/1875162680912...,https://at.linkedin.com/company/healcloud,https://healcloud.tech/,https://healcloud.tech,healcloud.tech,healcloud.tech
512,Rendity,https://www.eu-startups.com/directory/rendity/,https://www.eu-startups.com/wp-content/uploads...,https://rendity.com/,Vienna,Vienna,"finance, financial services, real estate",FinTech/InsurTech,Rendity enables real estate investments for ev...,2015,...,NaN,Rendity,Rendity,https://www.instagram.com/renditycom/,https://x.

## Get Google News via Serper dev

In [86]:
df = df_listings

In [ ]:
import os
import re
import time
import requests
import pandas as pd
from tqdm import tqdm
from dotenv import load_dotenv

# Configuration
load_dotenv()
API_KEY = os.getenv("SERPER_DEV_KEY")
if not API_KEY:
    raise RuntimeError("Please set SERPER_DEV_KEY in your .env file")

HEADERS = {
    "X-API-KEY": API_KEY
}
SEARCH_URL = "https://google.serper.dev/news"

# define your keyword list (lower‐case for easy matching)
KEYWORDS = [
    "startup", "finanzierung", "seed", "series a", "series b", "series c",
    "investment", "investor", "business angel", "kapital", "funding",
    "skalierung", "wachstum", "exit", "übernahme", "akquisition", "börsengang",
    "accelerator", "inkubator", "gründer", "founder", "start-up", "insolvenz",
    "restrukturierung", "beteiligung", "venture capital", "vc", "crowdfunding",
    "growth", "acquisition", "ipo"
]
# lower them once
KEYWORDS = [kw.lower() for kw in KEYWORDS]

# Helper to sanitize startup names
def make_safe(name: str) -> str:
    # strip out punctuation so we don’t trigger errors
    s = re.sub(r"[^\w\s]", " ", name)
    return re.sub(r"\s+", " ", s).strip()

def fetch_headlines(startup: str, n: int = 10, gl="AT", hl="de") -> list[dict]:
    safe = make_safe(startup)
    params = {
        "q":      f'"{safe}"',
        "source": "news",
        "gl":     gl,
        "hl":     hl
    }
    resp = requests.get(SEARCH_URL, headers=HEADERS, params=params)
    resp.raise_for_status()
    body = resp.json()
    items = body.get("news") or body.get("news_results") or []

    filtered = []
    for itm in items:
        title = itm.get("title", "")
        title_l = title.lower()

        # keep if the safe name appears
        match_name  = safe.lower() in title_l or startup.lower() in title_l
        # or any keyword appears
        match_kw   = any(kw in title_l for kw in KEYWORDS)

        if match_name or match_kw:
            filtered.append({"title": title, "link": itm.get("link")})
        if len(filtered) >= n:
            break

    return filtered


In [88]:
# Iterate and collect
results = []
for name in tqdm(df["name_cleaned"].dropna().unique(), desc="Fetching news"):
    try:
        news_items = fetch_headlines(name, n=20)
    except Exception as e:
        print(f"Error fetching news for {name!r}: {e}")
        news_items = []
    results.append({"name_cleaned": name, "Headlines": news_items})
    time.sleep(1)  # avoid rate limits

# ─── 5) Merge back and save ──────────────────────────────────────────────────
news_df = pd.DataFrame(results)

Fetching news: 100%|██████████| 515/515 [16:01<00:00,  1.87s/it]


In [93]:
merged_df = df.merge(
    news_df[['name_cleaned', 'Headlines']],  # only bring in the Headlines column
    on='name_cleaned',
    how='left'                               # keep all rows from df
)

merged_df.head(3)

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,age,total_funding,company_status,social_links,writing_score,clarity_score,innovativeness_score,market_readiness_score,founder_signal_score,sentiment_score,traction_score,word_count,sentence_count,jargon_density,numeric_evidence_count,call_to_action_flag,top_3_keywords,business_model,website_valid,website_clean,website_up,website_response_ms,redirected_url,name_cleaned,Startup,Instagram,X,LinkedIn,General,general_root,website_host,general_host,Headlines
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,5,No funding announced yet,Active,(),8,7,9.0,6,8,9,5,290,12,0.25,0,0,"['innovation', 'academic', 'entrepreneurial']",B2B,True,https://avdain.com,True,131.8,NaN,Avdain,Avdain,https://www.instagram.com/ns.rabbani/p/BSEJIDS...,https://x.com/awesomeindiehub/status/187271911...,NaN,https://www.avdain.com/our-mission,https://avdain.com,avdain.com,avdain.com,[]
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,3,Between €500K-€ 1 million,Active,"('https://www.linkedin.com/company/86313916',)",7,8,9.0,6,7,8,5,42,2,0.21,0,0,"['energy storage', 'EV charging', 'infrastruct...",B2B,True,https://popperpower.com,True,262.3,NaN,Popper Power,Popper Power,https://www.instagram.com/popperpower/,https://x.com/FranzHoegl/status/18789481448322...,https://at.linkedin.com/company/popperpower,https://www.popperpower.com/?srsltid=AfmBOoqrk...,https://popperpower.com,popperpower.com,popperpower.com,"[{'title': 'Insolvenz | Popper Power GmbH', 'l..."
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,1,Between €1-€100K,Active,(),8,9,7.0,8,6,9,7,305,15,0.20,2,1,"['sales', 'AI', 'integration']",B2B,True,https://trysetter.com,True,215.9,NaN,Setter AI,Setter AI,https://www.instagram.com/trysetter/,https://x.com/trysetter,https://www.linkedin.com/company/setterai?trk=...,https://www.trysetter.com/,https://trysetter.com,trysetter.com,trysetter.com,[{'title': 'Amazon-sjefen i Storbritannia sett...


## Validate Headlines via Chat GPT

In [108]:
import os
import re
import openai
import json
import pandas as pd
from dotenv import load_dotenv
from tqdm import tqdm

In [129]:
df = merged_df

In [ ]:
# Load keys
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    raise RuntimeError("Set OPENAI_API_KEY in your .env")
# instantiate the new client
client = openai.OpenAI(api_key=openai_api_key)

In [ ]:
# System message defining validity rules
system_message = {
    "role": "system",
    "content": (
        "You are a strict headline validator. A headline is VALID if it meets ANY of these:\n"
        "  • Mentions the startup by name,\n"
        "  • Announces a financing event (seed, Series A/B/C, investor, funding),\n"
        "  • Reports a risk event (insolvency, bankruptcy, layoffs, legal trouble),\n"
        "  • Describes a major milestone (product launch, partnership, exit, market expansion).\n"
        "All other headlines are INVALID."
    )
}

# Function spec with explicit guidance
validate_fn = {
    "name": "validate_headline",
    "description": (
        "Determine if a news headline is directly about the startup per the rules:\n"
        "– name mention, financing, risk event, milestone.\n"
        "Return valid=True/False and a short reason."
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The startup’s canonical name"
            },
            "headline": {
                "type": "string",
                "description": "The news headline text"
            },
            "valid": {
                "type": "boolean",
                "description": "True if on-topic per the rules"
            },
            "reason": {
                "type": "string",
                "description": "Brief justification"
            }
        },
        "required": ["name", "headline", "valid"]
    }
}

# Validation helper
def validate(startup: str, headline: str, location: str, keywords: list[str]):
    safe = make_safe(startup)
    user_msg = (
        f"Startup: {startup}\n"
        f"Location: {location}\n"
        f"Top keywords: {', '.join(keywords)}\n\n"
        f"Headline: {headline}\n\n"
        "Return JSON matching the validate_headline schema."
    )
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[system_message, {"role": "user", "content": user_msg}],
        functions=[validate_fn],
        function_call={"name": "validate_headline"}
    )
    args = json.loads(resp.choices[0].message.function_call.arguments)
    return {
        "name":     args["name"],
        "headline": args["headline"],
        "valid":    args["valid"],
        "reason":   args.get("reason", "")
    }

In [132]:
rows = []
for _, row in tqdm(df.iterrows(), total=len(df), desc="Validating"):
    name     = row["name_cleaned"]
    location = row.get("location", "Austria")
    keywords = row.get("top_3_keywords", []) or []
    for itm in row.get("Headlines", []) or []:
        title = itm.get("title")
        link  = itm.get("link")
        if not title:
            continue
        out = validate(name, title, location, keywords)
        out["link"] = link
        rows.append(out)
        time.sleep(0.5)

valid_df = pd.DataFrame(rows)

Validating: 100%|██████████| 515/515 [58:26<00:00,  6.81s/it]  


In [ ]:
# Filter, aggregate, and merge back
valid_df = valid_df[valid_df["valid"]]
counts = valid_df.groupby("name").size().rename("n_valid_headlines")
agg    = valid_df.groupby("name").agg({
    "headline": lambda s: list(s),
    "reason":   lambda s: list(s)
}).rename(columns={"headline":"valid_headlines","reason":"validation_reasons"})

final_df = (
    df
    .merge(counts, left_on="name_cleaned", right_index=True, how="left")
    .merge(agg,    left_on="name_cleaned", right_index=True, how="left")
)
final_df["n_valid_headlines"]      = final_df["n_valid_headlines"].fillna(0).astype(int)
final_df["valid_headlines"]        = final_df["valid_headlines"].apply(lambda x: x or [])
final_df["validation_reasons"]     = final_df["validation_reasons"].apply(lambda x: x or [])

In [137]:
final_df.head(3)

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,age,total_funding,company_status,social_links,writing_score,clarity_score,innovativeness_score,market_readiness_score,founder_signal_score,sentiment_score,traction_score,word_count,sentence_count,jargon_density,numeric_evidence_count,call_to_action_flag,top_3_keywords,business_model,website_valid,website_clean,website_up,website_response_ms,redirected_url,name_cleaned,Startup,Instagram,X,LinkedIn,General,general_root,website_host,general_host,Headlines,n_valid_headlines,valid_headlines,validation_reasons
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,5,No funding announced yet,Active,(),8,7,9.0,6,8,9,5,290,12,0.25,0,0,"['innovation', 'academic', 'entrepreneurial']",B2B,True,https://avdain.com,True,131.8,NaN,Avdain,Avdain,https://www.instagram.com/ns.rabbani/p/BSEJIDS...,https://x.com/awesomeindiehub/status/187271911...,NaN,https://www.avdain.com/our-mission,https://avdain.com,avdain.com,avdain.com,[],0,NaN,NaN
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,3,Between €500K-€ 1 million,Active,"('https://www.linkedin.com/company/86313916',)",7,8,9.0,6,7,8,5,42,2,0.21,0,0,"['energy storage', 'EV charging', 'infrastruct...",B2B,True,https://popperpower.com,True,262.3,NaN,Popper Power,Popper Power,https://www.instagram.com/popperpower/,https://x.com/FranzHoegl/status/18789481448322...,https://at.linkedin.com/company/popperpower,https://www.popperpower.com/?srsltid=AfmBOoqrk...,https://popperpower.com,popperpower.com,popperpower.com,"[{'title': 'Insolvenz | Popper Power GmbH', 'l...",4,"[Insolvenz | Popper Power GmbH, Insolvenz | He...","[Reports a risk event (insolvency)., Reports a..."
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,1,Between €1-€100K,Active,(),8,9,7.0,8,6,9,7,305,15,0.20,2,1,"['sales', 'AI', 'integration']",B2B,True,https://trysetter.com,True,215.9,NaN,Setter AI,Setter AI,https://www.instagram.com/trysetter/,https://x.com/trysetter,https://www.linkedin.com/company/setterai?trk=...,https://www.trysetter.com/,https://trysetter.com,trysetter.com,trysetter.com,[{'title': 'Amazon-sjefen i Storbritannia sett...,0,NaN,NaN


### Extract information from the headlines

From the success relevant headlines: extract information like: 
- got_funding: True / False
- funding amount 
- got_acquired: True / False
- bankrupcy: True / False

Non success relevant headlines + check if it is likely about an austrian startup, if not do not include with the name {}: 
- sentiment_score: 
- headline_count

In [140]:
import os
import json
import time
import pandas as pd
import openai
from dotenv import load_dotenv
from tqdm import tqdm

In [141]:
load_dotenv()
client = openai.OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [174]:
# ─── 1) Define the aggregate‐analysis function schema ───────────────────────
aggregate_fn = {
    "name": "analyze_startup_headlines",
    "description": (
        "Given a startup name and its validated headlines, "
        "split them into success‐relevant vs non‐success, then return:\n"
        "  • success_count (int) — how many success headlines (do include bankcrupy)\n"
        "  • total_funding_eur (number) — sum of all funding amounts mentioned\n"
        "  • acquisition_count (int)\n"
        "  • bankruptcy_count (int)\n"
        "  • non_success_count (int) — count of non‐success or financial impact (do not include bankruptcy) headlines\n"
    ),
    "parameters": {
        "type": "object",
        "properties": {
            "startup": {
                "type": "string",
                "description": "The startup’s canonical name"
            },
            "success_count": {
                "type": "integer",
                "description": "Number of headlines that report funding, M&A or bankruptcy"
            },
            "total_funding_eur": {
                "type": ["number","null"],
                "description": "Sum of all funding amounts mentioned in EUR"
            },
            "acquisition_count": {
                "type": "integer",
                "description": "Number of headlines reporting an acquisition"
            },
            "bankruptcy_count": {
                "type": "integer",
                "description": "Number of headlines reporting insolvency/bankruptcy"
            },
            "non_success_count": {
                "type": "integer",
                "description": "Number of the other headlines"
            },
        },
        "required": [
            "startup",
            "success_count",
            "total_funding_eur",
            "acquisition_count",
            "bankruptcy_count",
            "non_success_count"
        ]
    }
}

In [175]:
def aggregate_headlines(startup: str, headlines: list[str]):
    if not headlines:
        return {
            "startup": startup,
            "success_count": 0,
            "total_funding_eur": 0.0,
            "acquisition_count": 0,
            "bankruptcy_count": 0,
            "non_success_count": 0
        }

    list_block = "\n".join(f"- {h}" for h in headlines)
    user_msg   = f"Startup: {startup}\nHeadlines:\n{list_block}"

    # ← Move your arguments JSON inside function_call
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role":"system","content":"Aggregate these headlines into success vs non-success metrics."},
            {"role":"user","content":user_msg}
        ],
        functions=[aggregate_fn],
        function_call={
            "name": "analyze_startup_headlines",
            "arguments": json.dumps({
                "startup": startup,
                "results": [{"headline": h} for h in headlines]
            })
        }
    )

    out = json.loads(resp.choices[0].message.function_call.arguments)
    return out


In [181]:
df_analyze = final_df

In [182]:
df_analyze["valid_headlines"] = df_analyze["valid_headlines"].apply(
    lambda x: x if isinstance(x, list) else []
)

In [183]:
df_analyze.head(1)

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,age,total_funding,company_status,social_links,writing_score,clarity_score,innovativeness_score,market_readiness_score,founder_signal_score,sentiment_score,traction_score,word_count,sentence_count,jargon_density,numeric_evidence_count,call_to_action_flag,top_3_keywords,business_model,website_valid,website_clean,website_up,website_response_ms,redirected_url,name_cleaned,Startup,Instagram,X,LinkedIn,General,general_root,website_host,general_host,Headlines,n_valid_headlines,valid_headlines,validation_reasons
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,5,No funding announced yet,Active,(),8,7,9.0,6,8,9,5,290,12,0.25,0,0,"['innovation', 'academic', 'entrepreneurial']",B2B,True,https://avdain.com,True,131.8,NaN,Avdain,Avdain,https://www.instagram.com/ns.rabbani/p/BSEJIDS...,https://x.com/awesomeindiehub/status/187271911...,NaN,https://www.avdain.com/our-mission,https://avdain.com,avdain.com,avdain.com,[],0,[],NaN


In [184]:
# ─── 3) Run over your final_df, collect into features_df ────────────────────
rows = []
for _, row in tqdm(df_analyze.iterrows(), total=len(df_analyze), desc="Aggregating startups"):
    name      = row["name_cleaned"]
    headlines = row["valid_headlines"]   # now always a list
    metrics   = aggregate_headlines(name, headlines)
    rows.append(metrics)
    time.sleep(0.5)

features_df = pd.DataFrame(rows)

Aggregating startups: 100%|██████████| 515/515 [10:27<00:00,  1.22s/it]


In [186]:
features_df.head(3)

,startup,success_count,total_funding_eur,acquisition_count,bankruptcy_count,non_success_count
0,Avdain,0,0.0,0,0,0
1,Popper Power,0,0.0,0,4,0
2,Setter AI,0,0.0,0,0,0


In [195]:
# 1) Merge features back into your analysis DF
merged = df_analyze.merge(
    features_df,
    left_on="name_cleaned",
    right_on="startup",
    how="left"
)

In [196]:
# 2) Fill NaNs in all the numeric feature columns with 0
numeric_cols = [
    "success_count",
    "total_funding_eur",
    "acquisition_count",
    "bankruptcy_count",
    "non_success_count",
    "average_sentiment_non_success"
]
merged["total_funding_eur"] = merged["total_funding_eur"].fillna(0)

# 3) Convert total_funding_eur to integer
merged["total_funding_eur"] = merged["total_funding_eur"].astype(int)


In [197]:
merged.head(3)

,name,link_startupeu,link_logo,website,city,region,tags,category,business_description,founded,age,total_funding,company_status,social_links,writing_score,clarity_score,innovativeness_score,market_readiness_score,founder_signal_score,sentiment_score,traction_score,word_count,sentence_count,jargon_density,numeric_evidence_count,call_to_action_flag,top_3_keywords,business_model,website_valid,website_clean,website_up,website_response_ms,redirected_url,name_cleaned,Startup,Instagram,X,LinkedIn,General,general_root,website_host,general_host,Headlines,n_valid_headlines,valid_headlines,validation_reasons,startup,success_count,total_funding_eur,acquisition_count,bankruptcy_count,non_success_count
0,Avdain,https://www.eu-startups.com/directory/avdain/,https://www.eu-startups.com/wp-content/uploads...,avdain.com,Vienna,Vienna,"company, startup, one person",Software & Analytics,Avdain is a enterprise that embodies a fusion ...,2020,5,No funding announced yet,Active,(),8,7,9.0,6,8,9,5,290,12,0.25,0,0,"['innovation', 'academic', 'entrepreneurial']",B2B,True,https://avdain.com,True,131.8,NaN,Avdain,Avdain,https://www.instagram.com/ns.rabbani/p/BSEJIDS...,https://x.com/awesomeindiehub/status/187271911...,NaN,https://www.avdain.com/our-mission,https://avdain.com,avdain.com,avdain.com,[],0,[],NaN,Avdain,0,0,0,0,0
1,Popper Power GmbH,https://www.eu-startups.com/directory/popper-p...,https://www.eu-startups.com/wp-content/uploads...,www.popperpower.com,Vienna,Vienna,"ev, battery, bess, charging",Energy,Popper Power GmbH develops advanced energy sto...,2022,3,Between €500K-€ 1 million,Active,"('https://www.linkedin.com/company/86313916',)",7,8,9.0,6,7,8,5,42,2,0.21,0,0,"['energy storage', 'EV charging', 'infrastruct...",B2B,True,https://popperpower.com,True,262.3,NaN,Popper Power,Popper Power,https://www.instagram.com/popperpower/,https://x.com/FranzHoegl/status/18789481448322...,https://at.linkedin.com/company/popperpower,https://www.popperpower.com/?srsltid=AfmBOoqrk...,https://popperpower.com,popperpower.com,popperpower.com,"[{'title': 'Insolvenz | Popper Power GmbH', 'l...",4,"[Insolvenz | Popper Power GmbH, Insolvenz | He...","[Reports a risk event (insolvency)., Reports a...",Popper Power,0,0,0,4,0
2,Setter AI,https://www.eu-startups.com/directory/setter-ai/,https://www.eu-startups.com/wp-content/uploads...,https://www.trysetter.com,Wien,Vienna,"ai, ai agents, sales & marketing, ai saas, ai ...",Software & Analytics,Speed matters when you want more sales. That’s...,2024,1,Between €1-€100K,Active,(),8,9,7.0,8,6,9,7,305,15,0.20,2,1,"['sales', 'AI', 'integration']",B2B,True,https://trysetter.com,True,215.9,NaN,Setter AI,Setter AI,https://www.instagram.com/trysetter/,https://x.com/trysetter,https://www.linkedin.com/company/setterai?trk=...,https://www.trysetter.com/,https://trysetter.com,trysetter.com,trysetter.com,[{'title': 'Amazon-sjefen i Storbritannia sett...,0,[],NaN,Setter AI,0,0,0,0,0


In [198]:
merged.to_csv("./data/eustartup_listings_enriched_4.csv", index = False)

final_df